In [ ]:
!pip install transformers==4.43.1
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install huggingface_hub

In [ ]:
input_file = ""
output_file = ""

In [ ]:
from huggingface_hub import login

login(token='')

In [ ]:
import transformers
import torch

model_id = "google/gemma-2-9b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)

In [ ]:
def get_user_prompt(company, fr):
    return f'''###Contexto
Você é um analista financeiro com formação em economia que escreve para um público de investidores geral.

### Instrução:
Escreva em Português uma análise sobre a Empresa considerando o Fato Relevante e siga o Formato determinado. A análise deve conter TODOS os elementos especificados no seguinte Formato.

### Formato:
<Título>
<Sentença que Resuma a análise>
<Corpo da análise>
<Considerações Finais>

### Empresa:
{company}

### Fato Relevante:
{fr}

### Resposta: '''

In [ ]:
import pandas as pd

df = pd.read_csv(input_file)

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [ ]:
generated_texts = [None] * len(df)

In [ ]:
for index, row in df.tail(4).iterrows():

  empresa = row['empresa']
  fr_conteudo = row['FR_conteudo']

  instruct = get_user_prompt(empresa, fr_conteudo)

  messages = [
    {"role": "user", "content": instruct},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.3,
      top_p=0.4,
  )

  generated_texts[index] = outputs[0]["generated_text"][len(prompt):]

  del outputs
  del prompt
  del messages
  torch.cuda.empty_cache()
  gc.collect()


In [ ]:
df.loc[:len(generated_texts)-1, 'generated_text'] = generated_texts

In [ ]:
df.to_csv(output_file, index=False)